<a href="https://colab.research.google.com/github/ibribr/ML/blob/master/VGG16_Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from keras.datasets.mnist import load_data
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, Flatten
from keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from keras.applications.vgg16 import VGG16


In [ ]:
(xtrain, ytrain), (xtest, ytest) = load_data()
xtrain = xtrain.astype('float32') /255.
xtest = xtest.astype('float32') /255.
ytrain=to_categorical(ytrain)
ytest =to_categorical(ytest)

xtrain = xtrain.reshape(60000,28*28)
xtest = xtest.reshape(10000, 28*28)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(28*28, ), name='hidden_layer'))
model.add(Dense(10, activation='softmax', name='output_layer'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
plot_model(model, to_file='mnist_keras.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.fit(xtrain, ytrain, epochs=10, batch_size=128)
test_loss, test_accu = model.evaluate(xtest, ytest)
print(test_loss, test_accu)

In [ ]:
ypred = model.predict(xtest)

predicted_labels = np.argmax(ypred, axis=-1)
actual_labels = np.argmax(ytest, axis=-1)

print('Precision score:', precision_score(actual_labels, predicted_labels, average='macro'))
print('Recall score   :', recall_score(actual_labels, predicted_labels, average='macro'))


print('Confusion Matrix')
print(confusion_matrix(ytest.argmax(1), ypred.argmax(1)))

cmap = ListedColormap(['lightgrey', 'silver', 'ghostwhite', 'lavender', 'wheat'])

#confusion matrix
def cm(ytest, ypred, title):
  cm = confusion_matrix(ytest, ypred)

  plt.figure()
  plt.matshow(cm, cmap=cmap)
  
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      plt.text(x=j, y=i, s=cm[i,j], va='center', ha='center')
  
  plt.title(title)
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.show()

cm(ytrain.argmax(1), model.predict(xtrain).argmax(1), title='Train')
cm(ytest.argmax(1), model.predict(xtest).argmax(1), title='Test')

In [ ]:
# save your model: creates a h5 file `my_model.h5`.
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
reconstructed_model = load_model("my_h5_model.h5")

# test accuracy of this model
test_loss, test_accu = reconstructed_model.evaluate(xtest, ytest)
print(test_loss, test_accu)

In [ ]:
# test transfer learning: use a pre-trained model whos input shape is RGB of dim. (224, 224, 3) 

import cv2
import tensorflow as tf
from keras.datasets.mnist import load_data
import numpy as np
import matplotlib.pyplot as plt


(xtrain,ytrain), (xtest, ytest) = load_data()


width = int(224)
height = int(224)
dim = (width, height)

def rez(images):
  resized_images=[]
  for i in range(0,images.shape[0]):
    resized = cv2.resize(images[i], dim, interpolation = cv2.INTER_AREA)
    resized_images.append(resized)
  resized_images=np.asanyarray(resized_images)
  return resized_images

xtrain = rez(xtrain)
xtest = rez(xtest)

print(xtrain.shape)

plt.imshow(xtrain[0])
plt.show()

xtrain = xtrain.reshape((*xtrain.shape, 1))
print(xtrain.shape)


In [ ]:
# convert gray scale images into RGB

def gray2rgb(images):
  converted_imgs = []
  for i in range(0, images.shape[0]):  #images.shape[0]
    color_img = cv2.cvtColor(xtrain[i], cv2.COLOR_GRAY2RGB)
    converted_imgs.append(color_img)
  converted_imgs = np.asarray(converted_imgs)
  return converted_imgs

xtrain_conv = gray2rgb(xtrain)
print(xtrain_conv.shape)

plt.imshow(xtrain_conv[0])
plt.show()

In [ ]:
# Create the model
trans = Sequential()
trans.add(VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3)))
trans.add(Flatten())
trans.add(Dense(128, activation="relu"))
trans.add(Dense(64, activation="relu"))
trans.add(Dense(10, activation="sigmoid"))

trans.layers[0].trainable = False

trans.summary()
plot_model(trans, to_file='trans_mnist_keras.png', show_shapes=True, show_layer_names=True)

In [ ]:
trans.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
trans.fit(xtrain, ytrain, epochs=10, batch_size=128)
test_loss, test_accu = trans.evaluate(xtest, ytest)
print(test_loss, test_accu)
